<a href="https://colab.research.google.com/github/flaviohds/dota_analysis/blob/branch1/dota_update_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import time
from datetime import datetime
import pytz
from numpy import NaN

In [ ]:
!pip install selenium # mimics user clicks
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver #install chrome on the scrapper server

import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver

#chrome options that reduces bugs and variability
chrome_options = webdriver.ChromeOptions() 
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--start-maximized')
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36')
chrome_options.add_argument('--ignore-certificate-errors')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options, service_args=['--verbose', '--log-path=/tmp/chromedriver.log'])

In [ ]:
from google.colab import drive
drive.mount('/content/drive/') #mount drive on google drive

import os
os.chdir("drive/My Drive/dota") #map the folder on google drive

In [ ]:
#function updates heroes and abilities/spells/skills names from www.dota2.com/heroes
#updates up to hero_input
def update_hero_list(last_hero='ZEUS'):
  print('updating hero names')
  print('expected time: ~12mins')
  wd.delete_all_cookies()
  wd.get("https://www.dota2.com/hero/abaddon")
  wd.page_source;
  time.sleep(2)
  
  hero_list=[]
  is_last_hero=False
  spells_row=[]
  #loop to iterate heroes
  while is_last_hero==False:
    hero_name=wd.find_element('xpath','/html/body/div[2]/div/div/div[2]/div[5]/div[2]')

    if hero_name.text==last_hero:
      is_last_hero=True
    hero_row=[]
    hero_row.append(hero_name.text)

    j=0
    is_last_spell=False
    spell_row=[]
    #loop to iterate spells
    while is_last_spell==False:
      spell_name=wd.find_element('xpath','//*[@id="dota_react_root"]/div/div/div[4]/div[1]/div[2]/div[2]/div/div[1]/div/div[1]')
      spell_row.append(spell_name.text)

      try: #check if the next spell button exists
        ab_xpath='//*[@id="dota_react_root"]/div/div/div[4]/div[1]/div[2]/div[1]/div[2]/div['+str(j+2)+']'
        next_spell_b=wd.find_element('xpath',ab_xpath)
        next_spell_b.click()
      except: #if not, set the loop to end
        is_last_spell=True
      time.sleep(0.5)
      j+=1

    spell_row=list(dict.fromkeys(spell_row)) #removes duplicates
    hero_row.extend(spell_row)
    hero_list.append(hero_row)
    next_hero_b=wd.find_element('xpath','/html/body/div[2]/div/div/div[2]/div[2]/a[3]') #assign the next hero button
    next_hero_b.click() #click the next hero button
    time.sleep(1)

  df_hero_list=pd.DataFrame(hero_list) #creates the dataframe
  n_skills=df_hero_list.shape[1]-1 #checks the dimension of the dataframe to get the length of the hero with the highest number of spells
  columns_names=['hero'] #first column is the hero name
  for i in range(n_skills):
    columns_names.append('skill '+str(i+1)) #creates a list of columns names based with 'hero','skill 1','skill 2',...
  df_hero_list.columns=columns_names #attributes the columns names to the header
  df_hero_list.to_excel('hero_list.xlsx')

In [ ]:
#function updates the skills on the skills_list file
def update_skills_list():
  df_hero_list=pd.read_excel('hero_list.xlsx',index_col=0)
  skills_list=[]

  for i in df_hero_list.index: #iterates rows
    for j in df_hero_list.columns: #iterates columns
      if type(df_hero_list[j][i])==str: #checks if its a valid cell (not a NaN cell)
        skills_list.append(df_hero_list[j][i])
  df_skills_list=pd.DataFrame(skills_list)

  blacklist=['SPIRITS IN','SPIRITS OUT','ATTRIBUTE SHIFT (STRENGTH GAIN)'] #skills to remove from the list
  mask=df_skills_list.isin(blacklist)[0] #mask of where the blacklisted skills are ([0] to convert to a pandas.series)
  blacklist_indexes=df_skills_list[mask].index #indexes of where the blacklisted skills are
  df_skills_list=df_skills_list.drop(blacklist_indexes) #remove the blacklisted skills
  df_skills_list=df_skills_list.reset_index(drop=True)
  df_skills_list.to_excel('skills_list.xlsx')

In [ ]:
#function copies the attributes from the old skills_list file to the new one
#used when new skills/heroes are added
#make sure that the used dataframes do not have duplicated indexes (common when exporting and importing dataframes from files)
def update_skill_df(df_new, df_old, sort=True):
  df=df_new
  mask=df_new.isin(list(df_old[0])) #masks what skills are re-usable
  indexes_to_update=df_new[mask[0]].index #gets their indexes
  for i in indexes_to_update: #for each of the marked skills
    skill=df_new.loc[i,0] #checks the skill name
    row=df_old[df_old[0]==skill] #search the skill name and gets the row (with the attributes values) from the old dataframe
    row.index=[i] #corrects its index with the value in the new dataframe
    df=df.drop(i) #erases the "empty" row in the new dataframe
    df=pd.concat([df,row]) #ads the "filled" row to the new dataframe
  if sort==True: #not sorting the dataframe means the new spells (with pending attributes assignments) will all be at the top
    df=df.sort_index() #sorts the dataframe if wanted
  return df

In [ ]:
#function updates hero win and pickrates from www.dotabuff.com
def update_win_pick():
  print('updating win and pick rates')
  print('expected time: ~1min')
  wd.delete_all_cookies()
  wd.get("https://www.dotabuff.com/heroes/trends")
  wd.page_source;
  time.sleep(2)

  data_coord=[2,3,4,6,7,8]
  data=[]

  df_winpick=pd.read_excel('hero_list.xlsx',index_col=0,usecols=[1])
  total_heroes=df_winpick.shape[0]

  order_name_b=wd.find_element('xpath','/html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/thead/tr[3]/th[1]') #
  order_name_b.click()  #orders table by hero name

  for j in range(total_heroes):
    data_row=[]
    for i in range(len(data_coord)):
      xpath='/html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr['+str(j+1)+']/td['+str(data_coord[i])+']'
      html_element=wd.find_element('xpath',xpath)
      try:
        data_row+=[float(html_element.text.replace('%','').replace('+',''))] #removes % and + signs, converts to float and adds to the list
      except:
        data_row+=[0]

    data.append(data_row)

  df_winpick[['WIN_RATE_OLD','WIN_RATE','WIN_RATE_VAR','PICK_RATE_OLD','PICK_RATE','PICK_RATE_VAR']]=data

  # xpaths. last checked 23 june 2022
  # first hero
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[1]/td[2]  old_win
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[1]/td[3]  new_win
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[1]/td[4]  change_win
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[1]/td[6]  old_pick
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[1]/td[7]  new_pick
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[1]/td[8]  change_pick
  # second hero
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[2]/td[2]  old_win
  # /html/body/div[2]/div[2]/div[3]/div[4]/section/article/table/tbody/tr[2]/td[3]  new_win

  timestr=datetime.now(pytz.timezone('America/Sao_Paulo')).strftime('%Y%m%d') #time in BRT(GMT-3): year month day
  df_winpick.to_excel('winpick'+timestr+'.xlsx')

In [ ]:
# update_hero_list()
# update_skills_list()
update_win_pick()

In [ ]:
#cell to test update_skill_df
df_novo=pd.read_excel('skills_list_novo.xlsx',index_col=0)
df_velho=pd.read_excel('skills_list_velho.xlsx',index_col=0)

df_updated=update_skill_df(df_novo,df_velho,False)
df_updated.to_excel('skills_list_updated.xlsx')

In [ ]:
#to take a screenshot from chrome

# from PIL import Image
# from io import BytesIO

# figure=wd.get_screenshot_as_png()
# im = Image.open(BytesIO(figure))
# im